***GENERATED CODE FOR vehiclesales PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'make', 'transformation_label': 'String Indexer'}], 'feature': 'make', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '483', 'mean': '', 'stddev': '', 'min': 'Acura', 'max': 'Volvo', 'missing': '17', 'distinct': '36'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'make'}, {'feature_label': 'make', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('make')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'model', 'transformation_label': 'String Indexer'}], 'feature': 'model', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '483', 'mean': '1086.87', 'stddev': '1085.98', 'min': '1500', 'max': 'tC', 'missing': '17', 'distinct': '202'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'model'}, {'feature_label': 'model', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('model')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'trim', 'transformation_label': 'String Indexer'}], 'feature': 'trim', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '493', 'mean': '2.85', 'stddev': '0.49', 'min': '!', 'max': 'xDrive35d', 'missing': '7', 'distinct': '168'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'trim'}, {'feature_label': 'trim', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('trim')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'body', 'transformation_label': 'String Indexer'}], 'feature': 'body', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '492', 'mean': '', 'stddev': '', 'min': 'Convertible', 'max': 'wagon', 'missing': '17', 'distinct': '39'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'body'}, {'feature_label': 'body', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('body')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'transmission', 'transformation_label': 'String Indexer'}], 'feature': 'transmission', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '438', 'mean': '', 'stddev': '', 'min': 'automatic', 'max': 'manual', 'missing': '62', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'transmission'}, {'feature_label': 'transmission', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('transmission')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'vin', 'transformation_label': 'String Indexer'}], 'feature': 'vin', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '19uua86219a005187', 'max': 'zambb18a820006875', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'vin'}, {'feature_label': 'vin', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('vin')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'state', 'transformation_label': 'String Indexer'}], 'feature': 'state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ca', 'max': 'tx', 'missing': '0', 'distinct': '31'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'state'}, {'feature_label': 'state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('state')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'color', 'transformation_label': 'String Indexer'}], 'feature': 'color', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'beige', 'max': '�??', 'missing': '1', 'distinct': '14'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'color'}, {'feature_label': 'color', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('color')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'interior', 'transformation_label': 'String Indexer'}], 'feature': 'interior', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '', 'stddev': '', 'min': 'beige', 'max': '�??', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'interior'}, {'feature_label': 'interior', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('interior')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'seller', 'transformation_label': 'String Indexer'}], 'feature': 'seller', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1st commercial', 'max': 'young motors llc', 'missing': '0', 'distinct': '234'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'seller'}, {'feature_label': 'seller', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('seller')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'saledate', 'transformation_label': 'Extract Date'}], 'feature': 'saledate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'saledate'}, {'feature_label': 'saledate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('saledate')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run vehiclesalesHooks.ipynb
try:
	#sourcePreExecutionHook()

	carpricesdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/car_prices_data.csv', 'filename': 'car_prices_data.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(carpricesdata)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run vehiclesalesHooks.ipynb
try:
	#transformationPreExecutionHook()

	vehiclesalesautofe = TransformationMain.run(carpricesdata,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2011.3", "stddev": "2.91", "min": "2002", "max": "2015", "missing": "0"}, "updatedLabel": "year"}, {"transformationsData": [{"feature_label": "make", "transformation_label": "String Indexer"}], "feature": "make", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "483", "mean": "", "stddev": "", "min": "Acura", "max": "Volvo", "missing": "17", "distinct": "36"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "make"}, {"transformationsData": [{"feature_label": "model", "transformation_label": "String Indexer"}], "feature": "model", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "483", "mean": "1086.87", "stddev": "1085.98", "min": "1500", "max": "tC", "missing": "17", "distinct": "202"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "model"}, {"transformationsData": [{"feature_label": "trim", "transformation_label": "String Indexer"}], "feature": "trim", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "493", "mean": "2.85", "stddev": "0.49", "min": "!", "max": "xDrive35d", "missing": "7", "distinct": "168"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "trim"}, {"transformationsData": [{"feature_label": "body", "transformation_label": "String Indexer"}], "feature": "body", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "492", "mean": "", "stddev": "", "min": "Convertible", "max": "wagon", "missing": "17", "distinct": "39"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "body"}, {"transformationsData": [{"feature_label": "transmission", "transformation_label": "String Indexer"}], "feature": "transmission", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "438", "mean": "", "stddev": "", "min": "automatic", "max": "manual", "missing": "62", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "transmission"}, {"transformationsData": [{"feature_label": "vin", "transformation_label": "String Indexer"}], "feature": "vin", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "19uua86219a005187", "max": "zambb18a820006875", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "vin"}, {"transformationsData": [{"feature_label": "state", "transformation_label": "String Indexer"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ca", "max": "tx", "missing": "0", "distinct": "31"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "state"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "condition", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "27.4", "stddev": "13.95", "min": "1", "max": "49", "missing": "0"}, "updatedLabel": "condition"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "odometer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "499", "mean": "53763.7", "stddev": "34216.68", "min": "1", "max": "199924", "missing": "0"}, "updatedLabel": "odometer"}, {"transformationsData": [{"feature_label": "color", "transformation_label": "String Indexer"}], "feature": "color", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "beige", "max": "\ufffd??", "missing": "1", "distinct": "14"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "color"}, {"transformationsData": [{"feature_label": "interior", "transformation_label": "String Indexer"}], "feature": "interior", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "beige", "max": "\ufffd??", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "interior"}, {"transformationsData": [{"feature_label": "seller", "transformation_label": "String Indexer"}], "feature": "seller", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1st commercial", "max": "young motors llc", "missing": "0", "distinct": "234"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "seller"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mmr", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "17662.7", "stddev": "9404.13", "min": "975", "max": "71000", "missing": "0"}, "updatedLabel": "mmr"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sellingprice", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "11536.7", "stddev": "10483.51", "min": "250", "max": "76000", "missing": "0"}, "updatedLabel": "sellingprice"}, {"transformationsData": [{"feature_label": "saledate", "transformation_label": "Extract Date"}], "feature": "saledate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "saledate"}]}))

	#transformationPostExecutionHook(vehiclesalesautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run vehiclesalesHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(vehiclesalesautofe, ["year", "condition", "odometer", "mmr", "make_stringindexer", "model_stringindexer", "trim_stringindexer", "body_stringindexer", "transmission_stringindexer", "vin_stringindexer", "state_stringindexer", "color_stringindexer", "interior_stringindexer", "seller_stringindexer", "saledate_dayofmonth", "saledate_month", "saledate_year"], "sellingprice")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

